# EPA1361 - Model-Based Decision Making
# Week 3 - Sensitivity analysis

This exercise uses the same predator-prey model we used for the multi-model exercise, focusing on the Python version. As with the other exercise, define a model object for the function below, with the uncertainty ranges provided:

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |

* Sensitivity analysis often focuses on the final values of an outcome at the end of the simulation. However, we can also look at metrics that give us additional information about the behavior of the model over time. Using [the statsmodel library](https://www.statsmodels.org/stable/index.html) and an appropriate sampling design, fit a linear regression model for each of the following indicators. What can we conclude about the behavior of the model, and about the importance of the different inputs?

  * The final values of the _prey_ outcome
  * The mean values of the _prey_ outcome over time, within each experiment
  * The standard deviations of the _prey_ outcome over time, within each experiment
  

* Use the Sobol sampling functionality included in the Workbench to perform experiments with a sample size of N=50, then analyze the results with SALib for the same three indicators. This requires specifying the keyword argument `'uncertainty_sampling'` of perform_experiments. Note that when using Sobol sampling, the meaning of the keyword argument `scenarios` changes a bit. In order to properly estimate Sobol scores as well as interaction effects, you require N * (2D+2) scenarios, where D is the number of uncertain parameters, and N is the value for scenarios passed to `perform_experiments`. Repeat the analysis for larger sample sizes, with N=250 and N=1000. How can we interpret the first-order and total indices? Are these sample sizes sufficient for a stable estimation of the indices? You'll need to use the [get_SALib_problem](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/em_framework/salib_samplers.html) function to convert your Workbench experiments to a problem definition that you can pass to the SALib analysis function. 

* *hint*: sobol is a deterministic sequence of quasi random numbers. Thus, you can run with N=1000 and simply slice for 1:50 and 1:250.

* Use the [Extra-Trees analysis](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/analysis/feature_scoring.html) included in the Workbench to approximate the Sobol total indices, with a suitable sampling design. As a starting point, use an ensemble of 100 trees and a max_features parameter of 0.6, and set the analysis to regression mode. Are the estimated importances stable relative to the sample size and the analysis parameters? How do the results compare to the Sobol indices? For more details on this analysis see [Jaxa-Rozen & Kwakkel (2018)](https://www.sciencedirect.com/science/article/pii/S1364815217311581)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ema_workbench import (Model, MultiprocessingEvaluator, SequentialEvaluator, RealParameter, TimeSeriesOutcome, perform_experiments, ema_logging)

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

def pred_prey1(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25, 
             final_time=365, reps=1):

    #Initial values
    predators = np.zeros((reps, int(final_time/dt)+1))
    prey = np.zeros((reps, int(final_time/dt)+1))
    sim_time = np.zeros((reps, int(final_time/dt)+1))
    
    for r in range(reps):

        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

    #Calculate the time series
    for t in range(0, sim_time.shape[1]-1):

        dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
        dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

        prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
        predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
        sim_time[r,t+1] = (t+1)*dt
    
    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}

In [2]:
ema_logging.log_to_stderr(ema_logging.INFO)

# Setup the model
pred_prey = Model('PredPreyPy', function=pred_prey1)

pred_prey.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                      RealParameter('predator_efficiency', 0.0001, 0.004),
                      RealParameter('predator_loss_rate', 0.004, 0.08)]

pred_prey.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

In [3]:
from ema_workbench.em_framework.samplers import LHSSampler
from ema_workbench.em_framework.samplers import FullFactorialSampler as FFS

In [4]:
with SequentialEvaluator(pred_prey) as evaluator:
    experiment, outcomes = evaluator.perform_experiments(scenarios = 200, uncertainty_sampling=LHS)

[MainProcess/INFO] performing 200 scenarios * 1 policies * 1 model(s) = 200 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 60 cases completed
[MainProcess/INFO] 80 cases completed
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 120 cases completed
[MainProcess/INFO] 140 cases completed
[MainProcess/INFO] 160 cases completed
[MainProcess/INFO] 180 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] experiments finished


In [5]:
outcomes

{'TIME': array([[[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
          3.6475e+02, 3.6500e+02]],
 
        [[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
          3.6475e+02, 3.6500e+02]],
 
        [[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
          3.6475e+02, 3.6500e+02]],
 
        ...,
 
        [[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
          3.6475e+02, 3.6500e+02]],
 
        [[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
          3.6475e+02, 3.6500e+02]],
 
        [[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
          3.6475e+02, 3.6500e+02]]]),
 'predators': array([[[2.00000000e+01, 2.07191914e+01, 2.14632875e+01, ...,
          1.18535482e+02, 1.19054070e+02, 1.19521336e+02]],
 
        [[2.00000000e+01, 2.03612542e+01, 2.07293629e+01, ...,
          5.20928567e+01, 5.16919973e+01, 5.12901742e+01]],
 
        [[2.00000000e+01, 1.98305136e+01, 1.96615404e+01, ...,
          2.59309410e+00, 2.59483300e+00, 2.5

In [6]:
df = pd.DataFrame(outcomes['prey'][:,-1])
df['std'] = df.T.std()
df['last'] = df[1460]
df['mean'] = np.mean(df)

In [7]:
Y = df[['std', 'last', 'mean']]




In [17]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X = experiment[['predation_rate', 'predator_efficiency', 'predator_loss_rate', 'prey_birth_rate']]
scaled_data = scale.fit_transform(X)
X = pd.DataFrame(X)
X

,predation_rate,predator_efficiency,predator_loss_rate,prey_birth_rate
0,0.001269,0.003642,0.038270,0.021330
1,0.001466,0.002617,0.058575,0.031305
2,0.002823,0.000627,0.065224,0.032676
3,0.001794,0.002947,0.025650,0.022743
4,0.000878,0.000244,0.046014,0.029771
...,...,...,...,...
195,0.001376,0.002208,0.024688,0.021281
196,0.002440,0.000177,0.007735,0.032519
197,0.002949,0.000851,0.068487,0.027183
198,0.002648,0.001385,0.004519,0.015818


In [9]:
from statsmodels.regression.linear_model import OLS

In [14]:
regression_model = OLS(Y['std'], scaled_data).fit()
regression_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    std   R-squared (uncentered):                   0.170
Model:                            OLS   Adj. R-squared (uncentered):              0.153
Method:                 Least Squares   F-statistic:                              10.04
Date:                Mon, 10 May 2021   Prob (F-statistic):                    2.05e-07
Time:                        14:48:13   Log-Likelihood:                         -1010.9
No. Observations:                 200   AIC:                                      2030.
Df Residuals:                     196   BIC:                                      2043.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1981      2.713      0.073      0.942      -5.152       5.548
x2           -16.2639      2.715     -5.989      0.000     -21.619     -10.909
x3             3.1837      2.715      1.173      0.242      -2.170       8.538
x4             3.6826      2.718      1.355      0.177      -1.678       9.044
==============================================================================
Omnibus:                      197.096   Durbin-Watson:                   1.125
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4582.821
Skew:                           3.816   Prob(JB):                         0.00
Kurtosis:                      25.174   Cond. No.                         1.09
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
regression_model = OLS(Y['mean'], scaled_data).fit()
regression_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   mean   R-squared (uncentered):                   0.001
Model:                            OLS   Adj. R-squared (uncentered):             -0.019
Method:                 Least Squares   F-statistic:                            0.04436
Date:                Mon, 10 May 2021   Prob (F-statistic):                       0.996
Time:                        14:48:24   Log-Likelihood:                         -974.15
No. Observations:                 200   AIC:                                      1956.
Df Residuals:                     196   BIC:                                      1970.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.2295      2.257     -0.102      0.919      -4.681       4.222
x2            -0.4290      2.259     -0.190      0.850      -4.885       4.027
x3             0.8357      2.259      0.370      0.712      -3.619       5.290
x4            -0.1130      2.262     -0.050      0.960      -4.573       4.347
==============================================================================
Omnibus:                       33.116   Durbin-Watson:                   0.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               25.241
Skew:                           0.763   Prob(JB):                     3.30e-06
Kurtosis:                       2.164   Cond. No.                         1.09
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
regression_model = OLS(Y['last'], scaled_data).fit()
regression_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   last   R-squared (uncentered):                   0.215
Model:                            OLS   Adj. R-squared (uncentered):              0.199
Method:                 Least Squares   F-statistic:                              13.45
Date:                Mon, 10 May 2021   Prob (F-statistic):                    1.05e-09
Time:                        14:48:23   Log-Likelihood:                         -1128.6
No. Observations:                 200   AIC:                                      2265.
Df Residuals:                     196   BIC:                                      2278.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -1.8890      4.885     -0.387      0.699     -11.523       7.745
x2           -32.4664      4.890     -6.639      0.000     -42.110     -22.823
x3            15.5221      4.889      3.175      0.002       5.881      25.163
x4             0.7575      4.895      0.155      0.877      -8.897      10.412
==============================================================================
Omnibus:                      262.898   Durbin-Watson:                   1.417
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            17043.426
Skew:                           5.590   Prob(JB):                         0.00
Kurtosis:                      46.820   Cond. No.                         1.09
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
with SequentialEvaluator(pred_prey) as evaluator:
    experiment_SOBOL, outcomes_SOBOL = evaluator.perform_experiments(scenarios = 50, uncertainty_sampling=SOBOL)

[MainProcess/INFO] performing 500 scenarios * 1 policies * 1 model(s) = 500 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 50 cases completed
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 150 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 250 cases completed
[MainProcess/INFO] 300 cases completed
[MainProcess/INFO] 350 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 450 cases completed
[MainProcess/INFO] 500 cases completed
[MainProcess/INFO] experiments finished


In [22]:
df_SOBOL = pd.DataFrame(outcomes_SOBOL['prey'][:,-1])
df_SOBOL['std'] = df_SOBOL.T.std()
df_SOBOL['last'] = df_SOBOL[1460]
df_SOBOL['mean'] = np.mean(df_SOBOL)
Y_SOBOL = df_SOBOL[['std', 'last', 'mean']]

X_SOBOL = experiment_SOBOL[['predation_rate', 'predator_efficiency', 'predator_loss_rate', 'prey_birth_rate']]


In [23]:
SA_lib_problem = get_SALib_problem(pred_prey.uncertainties)

In [27]:
SA_lib_problem 

{'num_vars': 4,
 'names': ['predation_rate',
  'predator_efficiency',
  'predator_loss_rate',
  'prey_birth_rate'],
 'bounds': [(0.0005, 0.003), (0.0001, 0.004), (0.004, 0.08), (0.015, 0.035)]}

In [34]:
outcomes_SOBOL['prey'][:,-1]

array([[50.        , 50.09436035, 50.19018383, ..., 46.17815702,
        46.23564832, 46.29466654],
       [50.        , 50.05651855, 50.11457031, ..., 47.32230128,
        47.34925363, 47.37784027],
       [50.        , 50.09436035, 50.17979856, ...,  1.10645007,
         1.11273442,  1.1190704 ],
       ...,
       [50.        , 49.59942627, 49.17832151, ..., 55.4953864 ,
        55.59998155, 55.69756655],
       [50.        , 49.57427979, 49.12593697, ...,  0.46709588,
         0.46917647,  0.47126635],
       [50.        , 49.59942627, 49.17580106, ...,  0.82062189,
         0.8246892 ,  0.82877672]])

In [37]:
Si = sobol.analyze(SA_lib_problem, outcomes_SOBOL['prey'][:,-1] )

ValueError: could not broadcast input array from shape (50,1461) into shape (73050,)